# Bitmap - the binary datatype

**You can read the binary data structure for an image from a bitmap file. You can make changes to the image via the bitmap file.**

**In this exercise, read in a monochrome image of Halloween bat and invert the colours by writing updates back to the bitmap file. Yo unee to import the os module to access operating system requirements for processing bitmap files in Windows. Bitmap file formats change per operating system. The code below contains several steps to check whether the source file is actually a valid bitmap file.**

**NOTE: This is 3rd-party code, i.e. someone else wrote it! This exercise is more to understand the process, rather than the code, but make sure the source and test files are stored in the current working directory (not data folder). Otherwise the code will crash when writing the changes to the new 'inverted' file.** 

In [6]:
import os

source_file = 'vintage-halloween-bat.bmp'

# To test code handles files that are not bitmap (invalid)
#source_file = 'bm_test.txt'

inverted_file = f'inverted-{source_file}'

In [7]:
with open(source_file, 'rb') as bat:  # No encoding
    # Read the 14 byte header
    file_header = bat.read(14)

    bmp_id = file_header[0:2]
    print(bmp_id)
    if bmp_id == b'BM':  # If we have Windows bitmap file
        file_size = int.from_bytes(file_header[2:6], 'little')
        print(f'File size in header: {file_size}')
        os_size = os.path.getsize(source_file)
        print(f'File size reported by the operating system: {os_size}')
        
        # If file size meets operating system requirements, then it's bitmap
        if file_size != os_size:
            print('File size does not match file header size. '
                  'Are you sure this is a bitmap file?')
        else:
            reserved = file_header[6:10]
            print(f'For information, reserved bytes are: {reserved}')

            # Store when pixel data starts as an integer
            offset = int.from_bytes(file_header[-4:], 'little')
            print(f'Bitmap data starts at: {offset}')

            # Now read the DIB header and other information.
            # We're not interested in most of these values, but
            # we'll need them when writing the inverted file.
            # We read all bytes from the current position to 'offset'.
            dib_header_etc = bat.read(offset - bat.tell())

            # Check DIB header size
            dib_header_size = int.from_bytes(dib_header_etc[0:4], 'little')

            # We're only going to process BITMAPINFOHEADER files
            print(f'Bitmap header is {dib_header_size} bytes')
            if dib_header_size == 40:
                image_width = int.from_bytes(dib_header_etc[4:8], 'little', signed=True)
                image_height = int.from_bytes(dib_header_etc[8:12], 'little', signed=True)
                print(f'image is {image_width} by {image_height}')

                # Get the pixel data size (in bytes)
                pixel_array_size = int.from_bytes(dib_header_etc[20:24], 'little')
                print(f'Size of pixel array (bytes) = {pixel_array_size}')

                # Check: we should now be at 'offset' in the file.
                current_position = bat.tell()
                print(f'File pointer is at position {current_position}')
                if current_position != offset:
                    print(f"Something's gone wrong. We're at {current_position}, should be at {offset}")

                bat.seek(offset)  # Strictly speaking, this is redundant.

                # Read `pixel_array_size` bytes to get the image pixel data as bytearray
                image = bytearray(bat.read(pixel_array_size))
                
                # Invert image data by iterating over bytearray (mutable) 
                for index, bite in enumerate(image):
                    # Reverse the bits in each byte
                    image[index] = bite ^ 255

                # Now read the remainder of the file (if any)
                remainder = bat.read()
                
                # Open new inverted file for writing
                with open(inverted_file, 'wb') as inverted_bat:
                    print(f'\tWriting header')
                    inverted_bat.write(file_header)
                    print(f'\tWriting DIB header and other blocks')
                    inverted_bat.write(dib_header_etc)
                    print(f'\tWriting image data')
                    inverted_bat.write(image)
                    if remainder:
                        print(f'\tWriting remaining bytes')
                        inverted_bat.write(remainder)

                print(f'Image file {inverted_file} created.')
            else:
                print(f'{source_file} is not a supported bitmap format.')
    else:
        print(f'{source_file} does not appear to be a bitmap (.bmp) file.')


b'BM'
File size in header: 20862
File size reported by the operating system: 20862
For information, reserved bytes are: b'\x00\x00\x00\x00'
Bitmap data starts at: 62
Bitmap header is 40 bytes
image is 396 by 400
Size of pixel array (bytes) = 20800
File pointer is at position 62
	Writing header
	Writing DIB header and other blocks
	Writing image data
Image file inverted-vintage-halloween-bat.bmp created.


**A new bitmap file should appear in your working directory called 'inverted-vintage-halloween-bat.bmp '. You also have information on the file itself, like the correct 'BM' format and file size, and whether the file size in header is the same as reported by the operating system.**

**The pixel data starts at pointer position 62 (`offset` variable), which is used when working directly with the pixels. It is from this offset position that the pixel data is read in, and converted to bytearray to then invert the pixel data.**

**You can open the bitmap files in the IDE to view the inverted file along with the original source file. Technically, you should be able to process any bitmap file with this program, even colour images. You will get some interesting results though!**